In [1]:
import numpy as np 
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, LSTM

import string
from preprocessor import load_sonnets

sonnets = load_sonnets()

Using TensorFlow backend.


In [16]:
chars = list(string.ascii_lowercase)
#chars = list(string.ascii_letters)
chars += [",", ".", "?", "!", ":", ";", "'", "(", ")", " ", "\n", "-"]

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

train_strings = []
char_length = 41
char_skip = 3 #20

for _, s in enumerate(sonnets):
    #sonnet = "\n".join(sonnets[0]).lower()
    sonnet = "\n".join(s).lower()
    
    for i in range(0, len(sonnet) - char_length, char_skip):
        train_strings.append(sonnet[i: i+char_length])
    #break
    
x = np.zeros((len(train_strings), char_length - 1, len(chars)), dtype=np.bool)
y = np.zeros((len(train_strings), len(chars)), dtype=np.bool)

for i, train_string in enumerate(train_strings):
    for t, char in enumerate(train_string[:-1]):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[train_string[-1]]] = 1

In [17]:
#timesteps = 8
model = Sequential()
#model.add(Dense(200, input_shape=(200,)))
model.add(LSTM(200, input_shape=(char_length - 1, len(chars))))
#model.add(Dense(200))
#model.add(Activation('relu'))
#model.add(Dropout(dropout))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [18]:
#model.summary()
model.fit(x, y, batch_size=64, epochs=10)

Epoch 1/10
29325/29325 [==============================] - 75s 3ms/step - loss: 2.7564 - acc: 0.2334
Epoch 2/10
29325/29325 [==============================] - 80s 3ms/step - loss: 2.2884 - acc: 0.3380
Epoch 3/10
29325/29325 [==============================] - 76s 3ms/step - loss: 2.1240 - acc: 0.3771
Epoch 4/10
29325/29325 [==============================] - 72s 2ms/step - loss: 2.0084 - acc: 0.4093
Epoch 5/10
29325/29325 [==============================] - 70s 2ms/step - loss: 1.9258 - acc: 0.4290
Epoch 6/10
29325/29325 [==============================] - 72s 2ms/step - loss: 1.8573 - acc: 0.4478
Epoch 7/10
29325/29325 [==============================] - 72s 2ms/step - loss: 1.7966 - acc: 0.4612
Epoch 8/10
29325/29325 [==============================] - 80s 3ms/step - loss: 1.7417 - acc: 0.4754: 4s - 
Epoch 9/10
29325/29325 [==============================] - 74s 3ms/step - loss: 1.6867 - acc: 0.4900
Epoch 10/10
29325/29325 [==============================] - 74s 3ms/step - loss: 1.6343 - acc:

In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [28]:
def generate_poem(seed = "shall i compare thee to a summer's day?\n", lines=14):
    generated = "" + seed

    # Generates 14 lines of poem
    
    for i in range(lines):
        while True:
            x_pred = np.zeros((1, char_length - 1, len(chars)))
            for t, char in enumerate(seed):
                x_pred[0, t, char_indices[char]] = 1.
            preds = model.predict(x_pred, verbose=0)[0]
        
            next_index = sample(preds, temperature=0.5)
            next_char = indices_char[next_index]
            seed = seed[1:] + next_char
        
            generated += next_char
            if next_char == "\n":
                #print(generated)
                #generated = ""
                break
            #generated += next_char

    print(generated)
    
generate_poem()

shall i compare thee to a summer's day?
the beauty my seeftee the windst thee,
  the beest thou art of the gainty wert,
  the pairt the reast the seast thy love,
the prease the tore then the canlled,
and the tore-to ene thy wall thy sunded in
the sumer,
then a see the elest beautes that then the reast,
  for of the sull whe sone me prowing thines,
that me i me seade the seef the  im looks no now,
but the the the tround hand the tore then thee,
  the ofrect on the proust thy self to the prow,
that the tere, of the deer mayst is see.
  that i hall stain the seet the sweet in thee,
  the confore the hanger so thou soulded,



In [25]:
#seed = "shall i compare thee to a summer's day?\nthou art more lovely"
seed = "shall i compare thee to a summer's day?\n"
generated = "" + seed

# Generates 14 lines of poem (More efficient)

x_pred = np.zeros((1, char_length - 1, len(chars)))
for t, char in enumerate(seed):
    x_pred[0, t, char_indices[char]] = 1.
preds = model.predict(x_pred, verbose=0)[0]

for i in range(14):
    while True:
        next_index = sample(preds, temperature=0.5)
        next_char = indices_char[next_index]
        
        x_pred[0, :-1, :] = x_pred[:, 1:, :]
        x_pred[0, -1, :] = np.zeros(len(chars))
        x_pred[0, -1, char_indices[next_char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        
        generated += next_char
        if next_char == "\n":
            #print(generated)
            #generated = ""
            break
        #generated += next_char
        

print(generated)

shall i compare thee to a summer's day?
but the with thy beauthers my so for mine,
the days the rach thou thy self the rearted to the thee,
of the senfless chert of the love,
that in the parst the seare the wainst my seare.
  be your that i she hall still be of thee,
the forther then me that can the raysh thee thought,
the farse the reast bether that fer the still,
the looks my self-tored hand of tho gaik,
the hald with that it the paint the will,
o searing all i lave your my beauty,
the ald thy sulfer wathen thy self the see,
the thence,
the sur the parsed no brieg to tree,
  so more nom the seathen see the reart,



In [23]:
print(y.shape)
#print(generated)

(29325, 38)

